In [ ]:
import sys
import pandas as pd
import os
from cdcqr.common.config import LOCAL_DATA_DIR
sys.path.append('/core/github/cryptoderiv-quant/')
from ct.alfafactory import runsignal, cleaninorder
import time
from cdcqr.common.utils import play_sound
from IPython.display import Audio
sound_file = '/core/tmp/ding.wav'
import numpy as np
from cdcqr.data.glassnode import glassnode_data as gnd
import matplotlib.pyplot as plt
%matplotlib inline
from numba import jit

In [ ]:
gn = gnd.GlassnodeData()

In [ ]:
fs =  ['PriceUsdOhlc']
a = 'BTC'
df = gn.load_features(fs,a)
price0 = df.tail(1000).rename(columns={'BTC-PriceUsdOhlc-c':'c','BTC-PriceUsdOhlc-h':'h','BTC-PriceUsdOhlc-o':'o','BTC-PriceUsdOhlc-l':'l',})

In [ ]:
#price0 = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'btc1min.pkl'))

In [ ]:
price0.head(2)

In [ ]:
ma_ = price0['c'].rolling(50).mean()
std_ = price0['c'].rolling(50).std()
signal0 = price0['c'] > (ma_ + 2*std_)
signal0.mean()

In [ ]:
t0 = time.time()
n=200
ret = runsignal(price0.tail(n), signal0.tail(n), n1=None, n2=None)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
t0 = time.time()
n=200
ret = runsignal(price0.tail(n), signal0.tail(n), n1=None, n2=None)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

#### check runsignal function 

In [ ]:
n=1000
price = price0.tail(n).copy()
signal = signal0.tail(n).copy()

In [ ]:
price0

In [ ]:
tp=6
sl=2
ts=5
n1=1
n2=100
rsil=30
rsih=50
rsilag=14
maf=5
mas=15
short=False
size=np.inf
fees=0.07/100
freq='1h'
init_cash=10000
request=None

In [ ]:
signalclean=pd.Series.vbt.signals.empty_like(price['c'])

In [ ]:
signal.mean()

In [ ]:
res={}
ifigs=[]

runma=True
runrsi=True
runn=True
if maf is None or mas is None:
    runma=False
if rsil is None or rsih is None:
    runrsi=False
if n1 is None or n2 is None:
    runn=False

#clean price
price=price.resample(freq).last()

ifigs=[]


signalclean=pd.Series.vbt.signals.empty_like(price['c'])

In [ ]:
signalclean.loc[signal[signal].index]=True            

In [ ]:
signalclean.loc[signal[signal].index]=True            
signal=signalclean
signal=signal.ffill()

In [ ]:
#print("signal",signal)


SHORT=short

rsibelow=rsil
rsiabove=rsih
rsiperiod=rsilag

pf_kwargs = dict(size=size, fees=fees, freq=freq,init_cash=init_cash) #bidask spred 1bps    3bps cdc

fastmawindow=maf
slowmawindow=mas
#     signal=pd.Series.vbt.signals.empty_like(price['c'])
#     signal.loc[glass[glass['v'].shift(2)>700].index]=True

In [ ]:
signal.mean()

In [ ]:
entryn = n1  # entry after n periods
exitn = n2  # exit after n periods after entry
entriesn=signal.shift(entryn).fillna(False)
exitsnn=entriesn.shift(exitn).fillna(False)

In [ ]:
entriesn.mean()

In [ ]:
signal[signal].shape

In [ ]:
exitsnn[exitsnn].shape

In [ ]:
%%time
entriesnn_clean1,exitsnn_clean1=cleaninorder(entriesn,exitsnn)

In [ ]:
%%time
entriesnn_clean2,exitsnn_clean2=cleaninorder2(entriesn,exitsnn)

In [ ]:
display(entriesnn_clean1[entriesnn_clean1])
display(exitsnn_clean1[exitsnn_clean1])

In [ ]:
display(entriesnn_clean2[entriesnn_clean2])
display(exitsnn_clean2[exitsnn_clean2])

In [ ]:
exitsnn_clean[exitsnn_clean]

In [ ]:
def cleaninorder2(entriesn,exitsnn):
    x = pd.DataFrame()
    x['entries'] = entriesn
    x['exits'] = exitsnn
    x = x.astype(float)
    x['pos'] = x['entries'].replace(0,np.nan)
    # when entry is true we set exit to be false
    x['exits'].loc[x['entries']==1]=0
    s = x['exits'].eq(1).cumsum()
    x['pos'] = x.groupby(s)['pos'].ffill().fillna(0)
    diff = x['pos'].diff()
    newentries = diff==1
    newexits = diff==-1
    return newentries, newexits

In [ ]:
entriesnn_clean,exitsnn_clean=cleaninorder2(entriesn,exitsnn)

In [ ]:
entriesnn_clean[entriesnn_clean]

In [ ]:
exitsnn_clean[exitsnn_clean]

In [ ]:
entries = entriesn.copy()
exits = exitsnn.copy()
x = pd.DataFrame()
x['entries'] = entries
x['exits'] = exits
x['entrysignal'] = 1 * x['entries'] 
x['exitsignal']  = 1 * x['exits']
x['entrysignalsum'] = 0
x['entrysignalsum'][x['entrysignal']==1] = x['entrysignal'].cumsum()
x['exitsignalsum'] = 0
x['exitsignalsum'][x['exitsignal']==1] = x['exitsignal'].cumsum()
x['currenttrade'] = 0
for i in range(len(x)):
    # check current trade before, if zero fetch whatever entrysignal is 
    if (i == 0) or (x['currenttrade'][np.max([i-1,0])] == 0):
        x['currenttrade'][i] = x['entrysignalsum'][i]
    # otherwise check if current trade has ended now, --> no current trade
    elif x['currenttrade'][np.max([i-1,0])] == x['exitsignalsum'][i]:
        x['currenttrade'][i] = 0
    # else, current trade stays
    else:
        x['currenttrade'][i] = x['currenttrade'][i-1]
x['diff'] = x['currenttrade'].diff().fillna(x['currenttrade'][0])
x['newentries'] = (x['diff']>0)
x['newexits'] = (x['diff']<0)

In [ ]:
x[(x.index>='20220124') & (x.index<='20220130')].tail(140)

In [ ]:
@jit(parallel=True)
def cleaninorder(entries, exits):
    """
    
    cleans entries and exits where each exit corresponds to entry in order of appearence
    
    """
    x = pd.DataFrame()
    x['entries'] = entries
    x['exits'] = exits
    x['entrysignal'] = 1 * x['entries'] 
    x['exitsignal']  = 1 * x['exits']
    x['entrysignalsum'] = 0
    x['entrysignalsum'][x['entrysignal']==1] = x['entrysignal'].cumsum()
    x['exitsignalsum'] = 0
    x['exitsignalsum'][x['exitsignal']==1] = x['exitsignal'].cumsum()
    x['currenttrade'] = 0
    for i in range(len(x)):
        # check current trade before, if zero fetch whatever entrysignal is 
        if (i == 0) or (x['currenttrade'][np.max([i-1,0])] == 0):
            x['currenttrade'][i] = x['entrysignalsum'][i]
        # otherwise check if current trade has ended now, --> no current trade
        elif x['currenttrade'][np.max([i-1,0])] == x['exitsignalsum'][i]:
            x['currenttrade'][i] = 0
        # else, current trade stays
        else:
            x['currenttrade'][i] = x['currenttrade'][i-1]
    x['diff'] = x['currenttrade'].diff().fillna(x['currenttrade'][0])
    x['newentries'] = (x['diff']>0)
    x['newexits'] = (x['diff']<0)
    return x['newentries'],x['newexits']